In [1]:
import numpy as np
import os
import sys
import sqlite3 as sl

In [2]:
in_dir = "../db/"
out_file = "ProbAsn.db"

elems = ["H", "C", "N", "O", "S"]

# Create DB file

In [3]:
try:
    os.remove(f"{in_dir}{out_file}")
except:
    pass
con = sl.connect(f"{in_dir}{out_file}")

In [4]:
for e1 in elems:
    with con:
        con.execute(f"""
            CREATE TABLE {e1} (
                env VARCHAR(16),
                crystal VARCHAR(16),
                ind UNSIGNED INTEGER,
                shift FLOAT,
                err FLOAT,
                G2 VARCHAR(32),
                G3 VARCHAR(32),
                G4 VARCHAR(32),
                G5 VARCHAR(32),
                G6 VARCHAR(32)
            );
        """)
    for e2 in elems:
        with con:
            con.execute(f"""
                CREATE TABLE {e1}_{e2} (
                    env VARCHAR(16),
                    crystal VARCHAR(16),
                    ind UNSIGNED INTEGER,
                    shift FLOAT,
                    err FLOAT,
                    nei_ind UNSIGNED INTEGER,
                    nei_shift FLOAT,
                    nei_err FLOAT,
                    G2 VARCHAR(32),
                    G3 VARCHAR(32),
                    G4 VARCHAR(32),
                    G5 VARCHAR(32),
                    G6 VARCHAR(32)
                );
            """)

In [5]:
for f in  sorted(os.listdir(in_dir)):
    if f.endswith(".db") and f != out_file:
        
        print(f"Transferring database {f}")
        
        con2 = sl.connect(f"{in_dir}{f}")
        
        data = con2.execute(f"SELECT * FROM H")
        
        
        for e in elems:
            print(f"  Transferring {e}")
            # Transfer 1D graphs
            with con2:
                data = con2.execute(f"SELECT * FROM {e}")
                
            with con:
                for row in data:
                    env, crystal, ind, cs, err, h2, h3, h4, h5, h6 = row
                    con.execute(f"""INSERT INTO {e} (env, crystal, ind, shift, err, G2, G3, G4, G5, G6)
                                    VALUES ('{env}', '{crystal}', {ind}, {cs}, {err}, '{h2}', '{h3}', '{h4}', '{h5}', '{h6}')""")


            for e2 in elems:

                # Transfer 2D graphs
                with con2:
                    data = con2.execute(f"SELECT * FROM {e}_{e2}")
                    
                with con:
                    for row in data:
                        env, crystal, ind, cs, err, nei_ind, nei_cs, nei_err, h2, h3, h4, h5, h6 = row
                        con.execute(f"""INSERT INTO {e}_{e2} (env, crystal, ind, shift, err, nei_ind, nei_shift, nei_err, G2, G3, G4, G5, G6)
                                        VALUES ('{env}', '{crystal}', {ind}, {cs}, {err}, {nei_ind}, {nei_cs}, {nei_err},
                                        '{h2}', '{h3}', '{h4}', '{h5}', '{h6}')""")

Transferring database ProbAsn_AACMHX10-ACOJUZ.db
  Transferring H
  Transferring C
  Transferring N
  Transferring O
  Transferring S
Transferring database ProbAsn_ABEDOA-ZORQEC.db
  Transferring H
  Transferring C
  Transferring N
  Transferring O
  Transferring S
Transferring database ProbAsn_ACIJOM-OGOBAO.db
  Transferring H
  Transferring C
  Transferring N
  Transferring O
  Transferring S
Transferring database ProbAsn_ACOKAG-AFIQAI.db
  Transferring H
  Transferring C
  Transferring N
  Transferring O
  Transferring S
Transferring database ProbAsn_AFIQAJ-AJEBIA.db
  Transferring H
  Transferring C
  Transferring N
  Transferring O
  Transferring S
Transferring database ProbAsn_AJEBOG-AMDOAD.db
  Transferring H
  Transferring C
  Transferring N
  Transferring O
  Transferring S
Transferring database ProbAsn_AMDODP10-APIQIB.db
  Transferring H
  Transferring C
  Transferring N
  Transferring O
  Transferring S
Transferring database ProbAsn_APIQUM-ASUWOB01.db
  Transferring H
  Tran

In [7]:
con.close()